In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
import pyspark.sql.functions as F
import pandas as pd
import os
import gzip
import requests
from urllib.parse import urlparse
import json
from pyspark import SparkConf
from datetime import datetime
from pyspark.sql.types import *

StatementMeta(, 9c353ff1-e3a2-48c8-b5a3-52a888824dc6, 3, Finished, Available)

In [2]:
spark.conf.set('spark.sql.parquet.vorder.enabled', 'true')

filesnames = mssparkutils.fs.ls("abfss://workspace@onelake.dfs.fabric.microsoft.com/LakeHouse.Lakehouse/Files/folder")
# for files in filesnames:
#   print (files)

StatementMeta(, 9c353ff1-e3a2-48c8-b5a3-52a888824dc6, 4, Finished, Available)

In [8]:
def structure_structs(df):
    schema = df.schema
    for schema_element in schema:
        if type(schema_element.dataType) == StructType:
            for field in schema_element.dataType.fields:
                df = df.withColumn(f'{schema_element.name}_{field.name}',F.col(f'{schema_element.name}.{field.name}'))
            df = df.drop(schema_element.name)
            return df

def structure_arrays(df):
    schema = df.schema
    list_df = []
    for schema_element in schema:
        if type(schema_element.dataType) == ArrayType:
            df = df.withColumn(f"hash_key_{schema_element.name}",F.sha2(F.col(schema_element.name).cast("string"),256))
            new_df = df.select(f"hash_key_{schema_element.name}",schema_element.name)
            df = df.drop(schema_element.name)
            new_df = new_df.withColumn(schema_element.name,F.explode(F.col(schema_element.name)))
            list_df.append((schema_element.name,new_df))
    return df,list_df


StatementMeta(, 8aa87464-4b10-41c7-b55f-c0a14124e7f4, 10, Finished, Available)

In [18]:
# spark.sql("DROP TABLE NFPLakeHouse.jsond_in_network_negotiated_rates")
# spark.sql("DROP TABLE NFPLakeHouse.jsond_in_network")
# spark.sql("DROP TABLE NFPLakeHouse.jsond_in_network_negotiated_rates_negotiated_prices")
# spark.sql("DROP TABLE NFPLakeHouse.jsond_in_network_negotiated_rates_provider_references")
# spark.sql("DROP TABLE NFPLakeHouse.jsond_in_network_negotiated_rates_negotiated_prices_billing_code_modifier")
# spark.sql("DROP TABLE NFPLakeHouse.jsond_in_network_negotiated_rates_negotiated_prices_service_code")
# spark.sql("DROP TABLE NFPLakeHouse.jsond_main")
# spark.sql("DROP TABLE NFPLakeHouse.jsond_in_network_negotiated_rates_provider_groups_npi")
# spark.sql("DROP TABLE NFPLakeHouse.jsond_in_network_negotiated_rates_provider_groups")
# spark.sql("DROP TABLE NFPLakeHouse.jsond_provider_references")

StatementMeta(, 8aa87464-4b10-41c7-b55f-c0a14124e7f4, 20, Finished, Available)

DataFrame[]

In [9]:
for files in filesnames:
    #filename = files.name
    filename = 'Give the filename here'
    print (filename)
    df = spark.read.option("multiline","true").json(f'Files/folder/{filename}')
    
    done_dfs = []
    list_dfs = []
    #level = 0
    
    list_dfs.append(('main',df))

    while len(list_dfs) != 0:
        new_df_name, new_df = list_dfs.pop()

        while StructType in [type(schema_element.dataType) for schema_element in new_df.schema]:
            new_df = structure_structs(new_df)
        
        if ArrayType in [type(schema_element.dataType) for schema_element in new_df.schema]:
            new_df, new_dfs = structure_arrays(new_df)
            list_dfs.extend(new_dfs)
        done_dfs.append((new_df_name,new_df))
    break


StatementMeta(, 8aa87464-4b10-41c7-b55f-c0a14124e7f4, 11, Finished, Available)

2023-10_510_01B0_in-network-rates_11_of_29.json.gza5d906dc47486ea9f60885d85e95bb16a5c806ef98bb02efacc6baac9e3be0bc


In [10]:
for name,df in done_dfs:
    df.write.mode("append").option("mergeSchema","true").saveAsTable(f"LakeHouse.jsond_{name}")

StatementMeta(, 8aa87464-4b10-41c7-b55f-c0a14124e7f4, 12, Finished, Available)

In [12]:
df.printSchema()

StatementMeta(, 8aa87464-4b10-41c7-b55f-c0a14124e7f4, 14, Finished, Available)

root
 |-- hash_key_in_network_negotiated_rates_negotiated_prices_billing_code_modifier: string (nullable = true)
 |-- in_network_negotiated_rates_negotiated_prices_billing_code_modifier: string (nullable = true)



In [13]:
done_dfs

StatementMeta(, 8aa87464-4b10-41c7-b55f-c0a14124e7f4, 15, Finished, Available)

[('main',
  DataFrame[last_updated_on: string, reporting_entity_name: string, reporting_entity_type: string, version: string, hash_key_in_network: string]),
 ('in_network',
  DataFrame[hash_key_in_network: string, in_network_billing_code: string, in_network_billing_code_type: string, in_network_billing_code_type_version: string, in_network_description: string, in_network_name: string, in_network_negotiation_arrangement: string, hash_key_in_network_negotiated_rates: string]),
 ('in_network_negotiated_rates',
  DataFrame[hash_key_in_network_negotiated_rates: string, hash_key_in_network_negotiated_rates_negotiated_prices: string, hash_key_in_network_negotiated_rates_provider_groups: string]),
 ('in_network_negotiated_rates_provider_groups',
  DataFrame[hash_key_in_network_negotiated_rates_provider_groups: string, in_network_negotiated_rates_provider_groups_tin_type: string, in_network_negotiated_rates_provider_groups_tin_value: string, hash_key_in_network_negotiated_rates_provider_groups_